In [12]:
import numpy as np
np.random.seed(5) #Garantizamos volver a producir el entrenamiento

from keras.layers import Input, Dense, SimpleRNN
from keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from keras import backend as K

In [13]:
#Cargamos el dataset con los nombres
nombres = open('sample_data/nombre_dinosaurios.txt','r').read()
nombres = nombres.lower()

In [14]:
#Creamos una codificación onehot encoding para el entrenamiento

#Para ello creamos un alfabeto con todos los caractere 
#de nuestra lista de nombres
alfabeto = list(set(nombres))
tam_datos, tam_alfabeto = len(nombres), len(alfabeto)

print(alfabeto)

['g', 'f', 'v', 'k', 'b', 't', 'l', 'z', 'n', 'w', 'i', 's', 'p', 'e', 'm', 'u', 'c', 'y', 'h', 'a', 'q', 'x', 'd', 'j', 'r', 'o', '\n']


# Ejemplo de codificación onehot
Para la letra 'k' tendriamos la siguietne codificación

[e,\n,k,y,c,x,l,r,f,z,j,b,q,i,d,s,n,v,w,u,p,g,a,h,o,t,m]

[0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,]

El indice de la posición del caracter representará la letra.

In [15]:
#Ahora creamos los diccionariosa necesarios
char_to_ind = {char:ind for ind,char in enumerate(sorted(alfabeto))}

ind_to_char = {ind:char for ind,char in enumerate(sorted(alfabeto))}


# Creación de la arquitectura RNN

In [16]:
#Neuronas capa oculta
n_a = 25
#None nos permite nombres de entradas variables
input_layer = Input(shape=(None, tam_alfabeto))

layer_0 = Input(shape=(n_a,))

#Para que a la salida nos devuelva el nuevo estado oculto actualizado
recurrent_layer = SimpleRNN(n_a, activation='tanh', return_state=True)

output_layer = Dense(tam_alfabeto, activation='softmax')

hs, _ = recurrent_layer(input_layer, initial_state=layer_0)
output = []
output.append(output_layer(hs))
#Definimos las dos entradas, el caracter actual y el estado oculto anterior
model = Model([input_layer,layer_0], output)

opt = SGD(lr=0.0005)
model.compile(optimizer=opt, loss='categorical_crossentropy')


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


# Entrenamiento recurrente


In [18]:
with open("sample_data/nombre_dinosaurios.txt") as f:
    ejemplos = f.readlines()
ejemplos = [x.lower().strip() for x in ejemplos]
np.random.shuffle(ejemplos)

In [19]:
def train_generator():
    while True:
        # Tomar un ejemplo aleatorio
        ejemplo = ejemplos[np.random.randint(0,len(ejemplos))]

        # Convertir el ejemplo a representación numérica
        X = [None] + [char_to_ind[c] for c in ejemplo]

        # Crear "Y", resultado de desplazar "X" un caracter a la derecha
        Y = X[1:] + [char_to_ind['\n']]

        # Representar "X" y "Y" en formato one-hot
        x = np.zeros((len(X),1,tam_alfabeto))
        onehot = to_categorical(X[1:],tam_alfabeto).reshape(len(X)-1,1,tam_alfabeto)
        x[1:,:,:] = onehot
        y = to_categorical(Y,tam_alfabeto).reshape(len(X),tam_alfabeto)

        # Activación inicial (matriz de ceros)
        a = np.zeros((len(X), n_a))

        yield [x, a], y

In [20]:
BATCH_SIZE = 80 # Número de ejemplos de entrenamiento a usar en cada iteración
NITS = 10000 # Número de iteraciones

for j in range(NITS):
    historia = model.fit_generator(train_generator(), steps_per_epoch=BATCH_SIZE, epochs=1, verbose=0)

    # Imprimir evolución del entrenamiento cada 1000 iteraciones
    if j%1000 == 0:
        print('\nIteración: %d, Error: %f' % (j, historia.history['loss'][0]) + '\n')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """



Iteración: 0, Error: 3.287509


Iteración: 1000, Error: 2.376405


Iteración: 2000, Error: 2.242462


Iteración: 3000, Error: 2.166930


Iteración: 4000, Error: 2.195627


Iteración: 5000, Error: 2.148282


Iteración: 6000, Error: 2.126395


Iteración: 7000, Error: 2.226522


Iteración: 8000, Error: 2.183288


Iteración: 9000, Error: 2.157118



In [29]:
def generar_nombre(modelo,char_to_ind,tam_alfabeto,n_a):
    # Inicializar x y a con ceros
    x = np.zeros((1,1,tam_alfabeto,))
    a = np.zeros((1, n_a))

    # Nombre generado y caracter de fin de linea
    nombre_generado = ''
    fin_linea = '\n'
    car = -1

    # Iterar sobre el modelo y generar predicción hasta tanto no se alcance
    # "fin_linea" o el nombre generado llegue a los 50 caracteres
    contador = 0
    while (car != fin_linea and contador != 50):
          # Generar predicción usando la celda RNN
          a, _ = recurrent_layer(K.constant(x), initial_state=K.constant(a))
          y = output_layer(a)
          prediccion = K.eval(y)

          # Escoger aleatoriamente un elemento de la predicción (el elemento con
          # con probabilidad más alta tendrá más opciones de ser seleccionado)
          ix = np.random.choice(list(range(tam_alfabeto)),p=prediccion.ravel())

          # Convertir el elemento seleccionado a caracter y añadirlo al nombre generado
          car = ind_to_char[ix]
          nombre_generado += car

          # Crear x_(t+1) = y_t, y a_t = a_(t-1)
          x = to_categorical(ix,tam_alfabeto).reshape(1,1,tam_alfabeto)
          a = K.eval(a)

          # Actualizar contador y continuar
          contador += 1

          # Agregar fin de línea al nombre generado en caso de tener más de 50 caracteres
          if (contador == 10):
            nombre_generado += '\n'

    print(nombre_generado)

In [32]:
for i in range(10):
    generar_nombre(model,char_to_ind,tam_alfabeto,n_a)

ksadrarusa
bgsaycadhos

epheruruna
upaus

lusitralto
ruxausausallmaytishol

corusas

gaceriausa
usanauianhurisagtodphos

sausapusia
utofliankieptoptytaykarpcha

xhospbopel
evimerulus

papsataura
uromurapusaldhulusadiopisatrya

satmodurau
sauraiiachaschaustenpamiora

asipos

